# Prediction model

### Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
pd.set_option('display.max_columns', None)
df = pd.read_csv('fifa21_train.csv')

### Checking dataframe

In [2]:
df.rename(columns=str.lower, inplace=True)
df.head()

,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


### Selecting Y

In [3]:
y = df['ova']
df.drop('ova', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)

### Cleaning

In [4]:
def parse_height(height):
    feet, inches = height.split("'")
    return float(feet) + float(inches.replace('"', '')) / 12
def parse_weight(weight):
    return int(weight.replace('lbs', ''))
def parse_date(date_str):
    date = pd.to_datetime(date_str, errors='coerce')
    if pd.isna(date):
        return 0
    return int(date.strftime('%Y%m%d'))
def parse_value(value):
    if 'M' in value:
        return int(float(value.replace('€', '').replace('M', '')) * 1e6)
    elif 'K' in value:
        return int(float(value.replace('€', '').replace('K', '')) * 1e3)
    return int(value.replace('€', ''))
def sum_values(value):
    num1, num2 = value.split('+')
    return int(num1) + int(num2)

In [5]:
df['height'] = df['height'].apply(parse_height)
df['weight'] = df['weight'].apply(parse_weight)
df['joined'] = df['joined'].apply(parse_date)
df['value'] = df['value'].apply(parse_value)
df['wage'] = df['wage'].apply(parse_value)
df['release clause'] = df['release clause'].apply(parse_value)
df['foot'] = df['foot'].map({'Right': 1, 'Left': 0})

In [6]:
ratings_columns = ['w/f', 'sm', 'ir']
stats_columns = ['ls', 'gk', 'st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']  # Add all other stat columns here

# Convert ratings to numbers
for col in ratings_columns:
    df[col] = df[col].str.extract('(\d+)').astype(int)

def sum_values(value):
    # Check if 'value' is a string and contains a '+'
    if isinstance(value, str) and '+' in value:
        num1, num2 = value.split('+')
        return int(num1) + int(num2)
    # If 'value' is already an integer, return it as is
    elif isinstance(value, int):
        return value
    # Handle other cases, such as missing values or different data types
    else:
        return 0  # or a default value or appropriate handling for your case
for col in stats_columns:
    df[col] = df[col].apply(sum_values)

# Encode categorical columns with LabelEncoder
categorical_columns = ['nationality', 'club', 'bp', 'position']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store the label encoder for future use

# Drop unnecessary columns
df.drop(['name', 'team & contract', 'contract', 'loan date end'], axis=1, inplace=True)

# Convert a/w and d/w to distinct numbers
aw_dw_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
df['a/w'] = df['a/w'].map(aw_dw_mapping)
df['d/w'] = df['d/w'].map(aw_dw_mapping)


In [7]:
data_types =df.dtypes.reset_index()
data_types.columns=['Column', 'Type']
data_types[data_types['Type']=='object']

,Column,Type
67,hits,object


In [8]:
non_numeric=pd.to_numeric(df['hits'], errors='coerce').isna()

In [9]:
df['hits'][non_numeric]
df['hits'] = df['hits'].apply(parse_value)

In [10]:
df.head(60)

,age,nationality,club,bp,position,height,weight,foot,growth,joined,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,26,139,310,4,158,5.750000,161,1,1,20150701,525000,4000,801000,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,3.0,2.0,1,69,51,63,63,51,60,3,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,30,28,101,14,526,6.000000,159,1,0,20150116,8500000,23000,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,3.0,1.0,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,33,73,42,0,4,5.333333,134,1,0,20190131,9000000,49000,15300000,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,3.0,2.0,2,80,77,78,86,27,56,73,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,22,156,770,2,94,5.833333,152,1,13,20160701,275000,4000,694000,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,2.0,2.0,1,57,44,54,57,57,60,7,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,23,50,803,2,94,5.916667,150,1,8,20180701,725000,2000,1400000,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,1.0,2.0,1,66,44,60,64,60,66,4,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
5,33,28,721,7,272,5.833333,154,1,0,20200718,1400000,11000,2200000,328,69,64,64,72,59.0,333,71,62.0,64,64,72,358,73,74,75.0,69,67.0,321,65,63.0,70,64,59,300,56,38.0,68.0,74.0,64,71.0,103,31,34,38.0,65,16,10,15,8,16,1808,379,4,3,3.0,1.0,1,74,63,70,71,37,64,6,69,69,69,70,69,69,69,70,71,71,71,71,68,68,68,71,60,57,57,57,60,57,51,51,51,57,21
6,30,73,588,10,365,6.000000,154,1,0,20190902,6000000,28000,10800000,325,69,55,68,74,59.0,316,72,69.0,35,66,74,361,68,67,77.0,74,75.0,329,69,67.0,75,62,56,318,74,77.0,63.0,59.0,45,78.0,235,78,80,77.0,39,6,7,7,12,7,1923,410,5,3,2.0,3.0,2,67,58,67,73,77,68,49,68,68,68,68,67,67,67,68,70,70,70,71,72,72,72,71,76,76,76,76,76,76,76,76,76,76,17
7,22,73,12,7,1,5.833333,161,0,7,20190802,300000,800,540000,262,58,56,42,68,38.0,273,60,55.0,39,63,56,323,62,62,67.0,60,72.0,240,50,55.0,57,51,27,245,45,33.0,63.0,65.0,39,50.0,129,41,49,39.0,51,6,14,12,10,9,1523,327,2,2,2.0,2.0,1,62,48,63,60,42,52,4,57,57,57,59,58,58,58,59,62,62,62,63,61,61,61,63,54,54,54,54,54,52,48,48,48,52,17
8,34,17,312,1,89,6.333333,192,1,0,20180830,210000,2000,305000,277,54,46,70,63,44.0,281,56,43.0,57,62,63,200,33,35,34.0,64,34.0,343,54,86.0,57,90,56,294,62,69.0,62.0,52.0,49,63.0,192,58,69,65.0,49,15,6,7,9,12,1636,338,2,2,2.0,2.0,1,34,50,58,55,65,76,4,59,59,59,54,56,56,56,54,58,58,58,57,63,63,63,57,61,65,65,65,61,62,65,65,65,62,18
9,35,5,196,1,204,6.000000,163,0,0,20200127,220000,4000,319000,306,73,56,65,63,49.0,270,63,34.0,48,60,65,299,53,55,66.0,63,62.0,346,70,74.0,62,70,70,329,80,67.0,61.0,65.0,56,63.0,205,68,70,67.0,54,6,10,16,11,11,1809,381,3,3,2.0,2.0,1,54,62,63,64,68,70,3,64,64,64,63,62,62,62,63,65,65,65,64,66,66,66,64,67,6

### Clustering

In [11]:
# df.columns
cats = df[['nationality', 'club', 'bp', 'position','foot', 'growth','joined','w/f', 'sm', 'a/w', 'd/w', 'ir']]
nums = df[['age','value', 'wage', 'release clause','attacking', 'crossing', 'finishing', 'heading accuracy','short passing', 'volleys', 'skill', 'dribbling', 'curve','fk accuracy', 'long passing', 'ball control', 'movement','acceleration', 'sprint speed', 'agility', 'reactions', 'balance','power', 'shot power', 'jumping', 'stamina', 'strength', 'long shots','mentality', 'aggression', 'interceptions', 'positioning', 'vision','penalties', 'composure', 'defending', 'marking', 'standing tackle','sliding tackle', 'goalkeeping', 'gk diving', 'gk handling','gk kicking', 'gk positioning', 'gk reflexes', 'total stats','base stats','pac', 'sho', 'pas','dri', 'def', 'phy', 'hits', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf','rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm','cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']]

### Null values

In [12]:
null_values = df.isnull().sum().reset_index()
null_values.columns = ['Column', 'NullCount']
null_values = null_values[null_values['NullCount'] > 0]

null_values


,Column,NullCount
18,volleys,41
21,curve,41
28,agility,41
30,balance,41
33,jumping,41
39,interceptions,7
40,positioning,7
41,vision,41
43,composure,279
47,sliding tackle,41


In [13]:
df = df.fillna(0)

### Selecting features

In [14]:
import statsmodels.api as sm
X_added_constant = sm.add_constant(df)
X_added_constant
model = sm.OLS(y,X_added_constant).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ova   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     1665.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):               0.00
Time:                        09:50:11   Log-Likelihood:                -25046.
No. Observations:               11701   AIC:                         5.024e+04
Df Residuals:                   11629   BIC:                         5.077e+04
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.4304      1.730      4.874      0.000       5.040      11.821
age                  0.0529      0.009      5.666      0.000       0.035       0.071
nationality         -0.0003      0.000     -0.686      0.493      -0.001       0.001
club              7.638e-06   7.61e-05      0.100      0.920      -0.000       0.000
bp                  -0.0426      0.007     -6.389      0.000      -0.056      -0.030
position             0.0004      0.000      1.886      0.059   -1.44e-05       0.001
height              -0.6490      0.170     -3.821      0.000      -0.982      -0.316
weight               0.0060      0.002      2.722      0.006       0.002       0.010
foot                -0.1391      0.047     -2.950      0.003      -0.232      -0.047
growth              -0.1017      0.007    -13.688      0.000      -0.116      -0.087
joined            5.908e-07   7.11e-08      8.314      0.000    4.52e-07     7.3e-07
value             2.411e-07   1.95e-08     12.339      0.000    2.03e-07    2.79e-07
wage              3.077e-06   1.69e-06      1.825      0.068   -2.28e-07    6.38e-06
release clause   -4.488e-08   9.46e-09     -4.745      0.000   -6.34e-08   -2.63e-08
attacking            0.0154      0.004      3.811      0.000       0.007       0.023
crossing            -0.0315      0.006     -4.989      0.000      -0.044      -0.019
finishing           -0.0337      0.008     -4.119      0.000      -0.050      -0.018
heading accuracy     0.0918      0.006     16.190      0.000       0.081       0.103
short passing       -0.0071      0.008     -0.878      0.380      -0.023       0.009
volleys             -0.0040      0.004     -0.956      0.339      -0.012       0.004
skill               -0.0098      0.005     -2.146      0.032      -0.019      -0.001
dribbling           -0.0790      0.010     -8.222      0.000      -0.098      -0.060
curve                0.0276      0.005      5.635      0.000       0.018       0.037
fk accuracy          0.0280      0.005      6.002      0.000       0.019       0.037
long passing        -0.0427      0.006     -6.749      0.000      -0.055      -0.030
ball control         0.0562      0.008      7.323      0.000       0.041       0.071
movement             0.0396      0.004      9.813      0.000       0.032       0.047
acceleration        -0.0373      0.007     -5.686      0.000      -0.050      -0.024
sprint speed        -0.0478      0.008     -6.195      0.000      -0.063      -0.033
agility             -0.0388      0.005     -7.798      0.000      -0.049      -0.029
reactions            0.2013      0.006     33.712      0.000       0.190       0.213
balance             -0.0379      0.005     -8.241      0.000      -0.047      -0.029
power               -0.0043      0.003     -1.270      0.204      -0.011       0.002
shot power           0.

*P-values (P>|t|):*
This column shows the p-value for each feature’s coefficient.
A low p-value (typically < 0.05) indicates that you can reject the null hypothesis that the coefficient is zero (no effect).
Features with low p-values are generally considered more significant and are good candidates for inclusion in your model.

*Coefficients (coef):*
These values represent the estimated effect of each feature on the target variable.
Large coefficients imply a greater impact on the target variable, but significance is also determined by the p-value.

*R-squared (R²):*
This value indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.
A higher R-squared value means a better fit, but beware of overfitting with too many features.

*Adjusted R-squared:*
Similar to R-squared, but it adjusts for the number of features in the model.
It’s more reliable than R-squared for feature selection because it penalizes the addition of non-significant features.

*Confidence Interval (CI):*
The CI gives a range within which the true coefficient is likely to fall.
A wide interval may indicate that you need more data or that there is high variability in that feature’s effect.

*F-statistic:*
A higher F-statistic indicates that the model fits the data better than a model with no independent variables.
It’s used for testing the overall significance of the model.

*Durbin-Watson:*
This tests for the presence of autocorrelation in the residuals.
A value close to 2 suggests there is no autocorrelation.

*Condition Number:*
A high condition number indicates potential multicollinearity or numerical instability in the model.

In [15]:
df_sx = df.drop(columns=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'])
df_sx
#I dropped this categories as they are the summands of OVA, the target.

,age,nationality,club,bp,position,height,weight,foot,growth,joined,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits
0,26,139,310,4,158,5.750000,161,1,1,20150701,525000,4000,801000,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,3.0,2.0,1,69,51,63,63,51,60,3
1,30,28,101,14,526,6.000000,159,1,0,20150116,8500000,23000,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,3.0,1.0,2,83,75,68,82,33,71,44
2,33,73,42,0,4,5.333333,134,1,0,20190131,9000000,49000,15300000,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,3.0,2.0,2,80,77,78,86,27,56,73
3,22,156,770,2,94,5.833333,152,1,13,20160701,275000,4000,694000,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,2.0,2.0,1,57,44,54,57,57,60,7
4,23,50,803,2,94,5.916667,150,1,8,20180701,725000,2000,1400000,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,1.0,2.0,1,66,44,60,64,60,66,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,25,67,439,6,203,6.083333,168,0,5,20180208,625000,3000,891000,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3,2,2.0,2.0,1,75,28,56,59,60,69,3
11697,27,50,264,5,200,6.083333,174,1,4,20190701,1600000,4000,2900000,84,12,14,13,29,16.0,96,17,15.0,19,27,18,237,40,40,38.0,65,54.0,217,50,59.0,29,64,15,101,30,22.0,15.0,15.0,19,56.0,41,12,16,13.0,347,72,67,66,70,72,1123,387,2,1,2.0,2.0,1,72,67,66,72,40,70,3
11698,22,27,137,11,246,5.500000,143,1,8,20190123,1100000,2000,1700000,286,64,66,51,64,41.0,291,71,57.0,38,58,67,388,77,76,86.0,62,87.0,278,61,71.0,64,37,45,241,27,25.0,66.0,58.0,65,58.0,62,37,12,13.0,57,13,10,11,7,16,1603,337,3,3,3.0,2.0,1,76,60,60,72,26,43,6
11699,26,18,95,4,543,6.166667,179,1,7,0,0,0,0,254,64,66,51,73,0.0,284,74,0.0,63,72,75,198,71,72,0.0,55,0.0,258,61,0.0,71,64,62,241,64,61.0,63.0,0.0,53,0.0,113,55,58,0.0,81,9,24,72,24,24,1429,400,3,1,0.0,0.0,3,72,63,70,72,57,66,3


In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()

rfe = RFE(model, n_features_to_select=12)
rfe = rfe.fit(df_sx, y)

print(rfe.ranking_)

print(rfe.support_)

selected_features = df_sx.columns[rfe.support_]
print("Selected features:", selected_features)

[23 50 53  6 51  1 43  1  1 55 56 54 57 31 32 26  1  4 25 38  1 45 44  9
  1 21 22 20 10  1 19 37 35 46 49 30  3 28 42 36  1  7 27  8 52 11 47 41
 15 16 17 12 18 14 39  2 24  1 13  1  1 33 40 29  1  5 34 48]
[False False False False False  True False  True  True False False False
 False False False False  True False False False  True False False False
  True False False False False  True False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False  True False  True
  True False False False  True False False False]
Selected features: Index(['height', 'foot', 'growth', 'heading accuracy', 'dribbling',
       'ball control', 'reactions', 'positioning', 'sm', 'd/w', 'ir', 'dri'],
      dtype='object')


In [17]:
selected_features
#I will try these features mixes:
# 1 short version reducing possible overlapings
# 2 automatic selection
X1_ =df[['height', 'foot', 'positioning', 'growth','sm','d/w', 'ir','dri']]
X2_=df[selected_features]

### Scaling

In [18]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X1_)
x1_standardized = transformer.transform(X1_)
X1_scaled = pd.DataFrame(x1_standardized, columns=X1_.columns)

transformer = MinMaxScaler().fit(X2_)
x2_standardized = transformer.transform(X2_)
X2_scaled = pd.DataFrame(x2_standardized, columns=X2_.columns)

### Model

In [19]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y, test_size=0.25)
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(X1_train,y1_train)

from sklearn.metrics import r2_score
predictions1 = lm.predict(X1_train)
print(r2_score(y1_train, predictions1))
predictions_test1 = lm.predict(X1_test)
print(r2_score(y1_test, predictions_test1))

0.6840468896424545
0.6779408125060065


In [20]:
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y, test_size=0.25)
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(X2_train,y2_train)

from sklearn.metrics import r2_score
predictions2 = lm.predict(X2_train)
print(r2_score(y2_train, predictions2))
predictions_test2 = lm.predict(X2_test)
print(r2_score(y2_test, predictions_test2))

0.8657651269089156
0.8634535003812126


### Preliminary results

In this simple example for feature selection and linear regression, features in X2_ where the best to make predictions on OVA.

Further attemps may include:

- dividing the skills by position to avoid noise in case of goalkeepers
- try out other models for prediction, always carefully controlling overfitness


### y and x segmentation by position

In [21]:
bp = pd.read_csv('fifa21_train.csv')['BP']
bp.replace({'GK':'gk','CM':'mid','ST':'att','CAM':'mid','CDM':'mid','LM':'mid','RB':'def','CB':'def','RM':'mid','LB':'def','RW':'att','LWB':'def','LW':'att','RWB':'def','CF':'att'},inplace=True)
df_2 = pd.concat([df,bp],axis=1)
y_2 = pd.concat([y,bp],axis=1)
y_att = y_2.loc[y_2['BP']=='att']['ova']
y_defn = y_2.loc[y_2['BP']=='def']['ova']
y_midf = y_2.loc[y_2['BP']=='mid']['ova']
y_gk = y_2.loc[y_2['BP']=='gk']['ova']

In [22]:
X_gk = df_2.loc[df_2['BP']=='gk'].drop(columns=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','BP'],axis=1)

X_midf = df_2.loc[df_2['BP']=='mid'].drop(columns=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','BP'],axis=1)

X_defn = df_2.loc[df_2['BP']=='def'].drop(columns=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','BP'],axis=1)

X_att = df_2.loc[df_2['BP']=='att'].drop(columns=['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk','BP'],axis=1)


In [23]:
X_gk

,age,nationality,club,bp,position,height,weight,foot,growth,joined,value,wage,release clause,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits
16,28,54,329,5,200,6.250000,198,1,2,20191008,325000,3000,544000,101,19,16,18,28,20.0,86,20,14.0,19,16,17,246,47,49,33.0,61,56.0,200,43,41.0,41,63,12,103,20,16.0,12.0,33.0,22,25.0,48,18,17,13.0,307,62,60,57,62,66,1091,355,3,1,2.0,2.0,1,62,60,57,66,48,62,3
20,21,136,73,5,200,6.166667,163,1,8,20190701,350000,2000,805000,56,12,8,9,21,6.0,79,11,14.0,12,23,19,171,32,24,28.0,50,37.0,166,44,47.0,21,43,11,75,22,8.0,5.0,25.0,15,24.0,31,6,14,11.0,309,66,67,58,60,58,887,337,3,1,2.0,2.0,1,66,67,58,58,28,60,7
21,24,110,560,5,200,6.166667,176,1,7,20130304,1100000,2000,1600000,90,13,15,14,29,19.0,95,17,16.0,21,18,23,246,45,44,36.0,66,55.0,248,50,70.0,44,65,19,129,20,18.0,11.0,65.0,15,48.0,51,16,18,17.0,341,70,66,66,67,72,1200,385,3,1,2.0,2.0,1,70,66,66,72,44,67,9
62,23,136,136,5,200,6.000000,168,1,10,20180701,475000,4000,1100000,57,10,9,11,18,9.0,71,5,12.0,13,29,12,161,28,23,21.0,41,48.0,186,46,59.0,25,51,5,84,22,10.0,7.0,26.0,19,22.0,34,8,14,12.0,320,63,67,61,64,65,913,345,3,1,2.0,2.0,1,63,67,61,65,25,64,3
81,25,18,141,5,200,6.333333,176,1,5,20160729,4500000,7000,0,94,17,17,19,24,17.0,93,18,12.0,12,27,24,239,49,50,31.0,65,44.0,186,20,55.0,28,69,14,142,28,20.0,14.0,61.0,19,33.0,50,17,18,15.0,369,75,71,69,75,79,1173,419,2,1,2.0,2.0,1,75,71,69,79,50,75,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11670,37,73,434,5,543,6.083333,176,1,0,20170705,45000,8000,77000,77,11,15,10,28,13.0,77,8,14.0,13,27,15,189,29,29,37.0,59,35.0,172,44,43.0,26,43,16,96,25,13.0,10.0,28.0,20,58.0,27,3,12,12.0,313,64,60,59,64,66,951,342,2,1,2.0,2.0,1,64,60,59,66,29,64,3
11675,22,103,335,5,200,6.250000,181,1,10,20180701,2300000,6000,3900000,68,13,10,13,23,9.0,86,14,12.0,10,27,23,201,31,32,35.0,71,32.0,205,49,58.0,35,55,8,89,19,12.0,11.0,37.0,10,36.0,38,12,12,14.0,340,69,66,65,69,71,1027,372,3,1,2.0,2.0,1,69,66,65,71,32,69,22
11682,31,28,781,5,200,6.333333,185,1,0,20160112,240000,3000,372000,69,15,12,17,13,12.0,100,17,12.0,15,34,22,246,62,45,56.0,60,23.0,222,53,56.0,37,65,11,122,32,13.0,13.0,45.0,19,63.0,61,20,17,24.0,319,61,59,70,59,70,1139,372,1,1,2.0,2.0,1,61,59,70,70,53,59,3
11688,27,50,528,5,200,6.000000,190,1,3,20100701,650000,7000,1300000,83,13,10,14,36,10.0,85,10,10.0,13,34,18,232,46,42,37.0,65,42.0,196,25,68.0,39,54,10,78,13,16.0,14.0,22.0,13,53.0,36,12,13,11.0,330,70,65,64,63,68,1040,374,3,1,2.0,2.0,1,70,65,64,68,44,63,3


In [24]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()

rfe = RFE(model, n_features_to_select=12)
rfe.fit(X_gk, y_gk)  # For goalkeepers
selected_features_gk = X_gk.columns[rfe.support_]

rfe.fit(X_midf, y_midf)  # For midfielders
selected_features_midf = X_midf.columns[rfe.support_]

rfe.fit(X_defn, y_defn)  # For defenders
selected_features_defn = X_defn.columns[rfe.support_]

rfe.fit(X_att, y_att)  # For attackers
selected_features_att = X_att.columns[rfe.support_]


In [25]:
print(selected_features_midf)
print(selected_features_defn)
print(selected_features_att)
print(selected_features_gk)

Index(['height', 'skill', 'curve', 'fk accuracy', 'long passing',
       'ball control', 'reactions', 'a/w', 'd/w', 'ir', 'pas', 'phy'],
      dtype='object')
Index(['bp', 'height', 'foot', 'reactions', 'base stats', 'sm', 'a/w', 'd/w',
       'ir', 'sho', 'pas', 'def'],
      dtype='object')
Index(['bp', 'height', 'foot', 'finishing', 'ball control', 'reactions',
       'shot power', 'positioning', 'd/w', 'ir', 'dri', 'def'],
      dtype='object')
Index(['gk diving', 'gk handling', 'gk positioning', 'gk reflexes',
       'base stats', 'ir', 'pac', 'sho', 'pas', 'dri', 'def', 'phy'],
      dtype='object')


### New model

In [26]:
selected_features_gk = X_gk[selected_features_gk]
selected_features_defn = X_defn[selected_features_defn]
selected_features_midf = X_midf[selected_features_midf]
selected_features_att = X_att[selected_features_att]


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Split your data into training and testing sets
X_train_att, X_test_att, y_train_att, y_test_att = train_test_split(selected_features_att, y_att, test_size=0.2, random_state=42)
X_train_defn, X_test_defn, y_train_defn, y_test_defn = train_test_split(selected_features_defn, y_defn, test_size=0.2, random_state=42)
X_train_midf, X_test_midf, y_train_midf, y_test_midf = train_test_split(selected_features_midf, y_midf, test_size=0.2, random_state=42)
X_train_gk, X_test_gk, y_train_gk, y_test_gk = train_test_split(selected_features_gk, y_gk, test_size=0.2, random_state=42)

# the linear regression models
model_att = LinearRegression().fit(X_train_att, y_train_att)
model_defn = LinearRegression().fit(X_train_defn, y_train_defn)
model_midf = LinearRegression().fit(X_train_midf, y_train_midf)
model_gk = LinearRegression().fit(X_train_gk, y_train_gk)

# Make predictions on the test set
y_pred_att = model_att.predict(X_test_att)
y_pred_defn = model_defn.predict(X_test_defn)
y_pred_midf = model_midf.predict(X_test_midf)
y_pred_gk = model_gk.predict(X_test_gk)

# Calculate R² scores
r2_att = r2_score(y_test_att, y_pred_att)
r2_defn = r2_score(y_test_defn, y_pred_defn)
r2_midf = r2_score(y_test_midf, y_pred_midf)
r2_gk = r2_score(y_test_gk, y_pred_gk)

print(f"R² score for attackers: {r2_att:.4f}")
print(f"R² score for defenders: {r2_defn:.4f}")
print(f"R² score for midfielders: {r2_midf:.4f}")
print(f"R² score for goalkeepers: {r2_gk:.4f}")


R² score for attackers: 0.9630
R² score for defenders: 0.9614
R² score for midfielders: 0.9211
R² score for goalkeepers: 0.9926


# Results
Segmentation, features selection and random forest resulted to be a good way to make the predictions accurate and symmetrical accross segments